In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
# Import Python libraries
import logging

# Import Pyomo units
from pyomo.environ import units as pyunits
from pyomo.environ import SolverFactory

from idaes.core import FlowsheetBlock

# Import IDAES cores
from idaes.core import LiquidPhase, VaporPhase, Component

from idaes.generic_models.properties.core.state_definitions import FTPx
from idaes.generic_models.properties.core.eos.ideal import Ideal
from idaes.generic_models.properties.core.phase_equil import SmoothVLE
from idaes.generic_models.properties.core.phase_equil.bubble_dew import \
        IdealBubbleDew,LogBubbleDew
from idaes.generic_models.properties.core.eos.ceos import Cubic, CubicType
from idaes.generic_models.properties.core.phase_equil.forms import fugacity,log_fugacity

from idaes.generic_models.properties.core.pure import RPP4
from idaes.generic_models.properties.core.pure import Perrys

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

In [3]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [4]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [5]:
data = pd.read_csv('BT_NRTL_dataset.csv')
print(data)

    temperature  liq_benzene  vap_benzene
0    365.500000     0.480953     0.692110
1    365.617647     0.462444     0.667699
2    365.735294     0.477984     0.692441
3    365.852941     0.440547     0.640336
4    365.970588     0.427421     0.623328
5    366.088235     0.442725     0.647796
6    366.205882     0.434374     0.637691
7    366.323529     0.444642     0.654933
8    366.441176     0.427132     0.631229
9    366.558824     0.446301     0.661743
10   366.676471     0.438004     0.651591
11   366.794118     0.425320     0.634814
12   366.911765     0.439435     0.658047
13   367.029412     0.435655     0.654539
14   367.147059     0.401350     0.604987
15   367.264706     0.397862     0.601703
16   367.382353     0.415821     0.630930
17   367.500000     0.420667     0.640380
18   367.617647     0.391683     0.598214
19   367.735294     0.404903     0.620432
20   367.852941     0.409563     0.629626
21   367.970588     0.389488     0.600722
22   368.000000     0.396789     0

### 2.3 Creating a generic property parameter block

The next step is to give all the parameter information required for the sub-libraries. Here we will specify the base units of our model, the bounds on the state properties and the reference conditions of our properties.

In [6]:
# Add properties parameter blocks to the flowsheet with specifications -- pull from .py file
# from idaes.generic_models.properties.core.examples.BT_VDW import configuration

In [7]:
#Make configuration from original Dictionary_Txy_diagrams_AG
configuration = {
    # Specifying components
    "components": {
        'benzene': {"type": Component,
                    "elemental_composition": {"C": 6, "H": 6},
                    "enth_mol_liq_comp": Perrys,
                    "enth_mol_ig_comp": RPP4,
                    "pressure_sat_comp": RPP4,
                    "phase_equilibrium_form": {("Vap", "Liq"): fugacity},
                    "parameter_data": {
                        "mw": (78.1136E-3, pyunits.kg/pyunits.mol),  # [1]
                        "pressure_crit": (48.9e5, pyunits.Pa),  # [1]
                        "temperature_crit": (562.2, pyunits.K),  # [1]
                        "omega": 0.212,  # [1]
                        "dens_mol_liq_comp_coeff": {
                            '1': (1.0162, pyunits.kmol*pyunits.m**-3),  # [2] pg. 2-98
                            '2': (0.2655, None),
                            '3': (562.16, pyunits.K),
                            '4': (0.28212, None)},
                        "cp_mol_ig_comp_coeff": {
                            'A': (-3.392E1, pyunits.J/pyunits.mol/pyunits.K),  # [1]
                            'B': (4.739E-1, pyunits.J/pyunits.mol/pyunits.K**2),
                            'C': (-3.017E-4, pyunits.J/pyunits.mol/pyunits.K**3),
                            'D': (7.130E-8, pyunits.J/pyunits.mol/pyunits.K**4)},
                        "cp_mol_liq_comp_coeff": {
                            '1': (1.29E2, pyunits.J/pyunits.kmol/pyunits.K),  # [2]
                            '2': (-1.7E-1, pyunits.J/pyunits.kmol/pyunits.K**2),
                            '3': (6.48E-4, pyunits.J/pyunits.kmol/pyunits.K**3),
                            '4': (0, pyunits.J/pyunits.kmol/pyunits.K**4),
                            '5': (0, pyunits.J/pyunits.kmol/pyunits.K**5)},
                        "enth_mol_form_liq_comp_ref": (
                            49.0e3, pyunits.J/pyunits.mol),  # [3]
                        "enth_mol_form_vap_comp_ref": (
                            82.9e3, pyunits.J/pyunits.mol),  # [3]
                        "pressure_sat_comp_coeff": {'A': (-6.98273, None),  # [1]
                                                    'B': (1.33213, None),
                                                    'C': (-2.62863, None),
                                                    'D': (-3.33399, None)}}},
        'toluene': {"type": Component,
                    "elemental_composition": {"C": 7, "H": 8},
                    "enth_mol_liq_comp": Perrys,
                    "enth_mol_ig_comp": RPP4,
                    "pressure_sat_comp": RPP4,
                    "phase_equilibrium_form": {("Vap", "Liq"): fugacity},
                    "parameter_data": {
                        "mw": (92.1405E-3, pyunits.kg/pyunits.mol),  # [1]
                        "pressure_crit": (41e5, pyunits.Pa),  # [1]
                        "temperature_crit": (591.8, pyunits.K),  # [1]
                        "omega": 0.263,  # [1]
                        "dens_mol_liq_comp_coeff": {
                            '1': (0.8488, pyunits.kmol*pyunits.m**-3),  # [2] pg. 2-98
                            '2': (0.26655, None),
                            '3': (591.8, pyunits.K),
                            '4': (0.2878, None)},
                        "cp_mol_ig_comp_coeff": {
                            'A': (-2.435E1, pyunits.J/pyunits.mol/pyunits.K),  # [1]
                            'B': (5.125E-1, pyunits.J/pyunits.mol/pyunits.K**2),
                            'C': (-2.765E-4, pyunits.J/pyunits.mol/pyunits.K**3),
                            'D': (4.911E-8, pyunits.J/pyunits.mol/pyunits.K**4)},
                        "cp_mol_liq_comp_coeff": {
                            '1': (1.40E2, pyunits.J/pyunits.kmol/pyunits.K),  # [2]
                            '2': (-1.52E-1, pyunits.J/pyunits.kmol/pyunits.K**2),
                            '3': (6.95E-4, pyunits.J/pyunits.kmol/pyunits.K**3),
                            '4': (0, pyunits.J/pyunits.kmol/pyunits.K**4),
                            '5': (0, pyunits.J/pyunits.kmol/pyunits.K**5)},
                        "enth_mol_form_liq_comp_ref": (
                            12.0e3, pyunits.J/pyunits.mol),  # [3]
                        "enth_mol_form_vap_comp_ref": (
                            50.1e3, pyunits.J/pyunits.mol),  # [3]
                        "pressure_sat_comp_coeff": {'A': (-7.28607, None),  # [1]
                                                    'B': (1.38091, None),
                                                    'C': (-2.83433, None),
                                                    'D': (-2.79168, None)}}}},

    # Specifying phases
    "phases":  {'Liq': {"type": LiquidPhase,
                        "equation_of_state": Cubic,
                        "equation_of_state_options": {
                            "type": CubicType.VDW}},
                'Vap': {"type": VaporPhase,
                        "equation_of_state": Cubic,
                        "equation_of_state_options": {
                            "type": CubicType.VDW}}},

    # Set base units of measurement
    "base_units": {"time": pyunits.s,
                   "length": pyunits.m,
                   "mass": pyunits.kg,
                   "amount": pyunits.mol,
                   "temperature": pyunits.K},

    # Specifying state definition
    "state_definition": FTPx,
    "state_bounds": {"flow_mol": (0, 100, 1000, pyunits.mol/pyunits.s),
                     "temperature": (273.15, 300, 450, pyunits.K),
                     "pressure": (5e4, 1e5, 1e6, pyunits.Pa)},
    "pressure_ref": (1e5, pyunits.Pa),
    "temperature_ref": (300, pyunits.K),

    # Defining phase equilibria
    "phases_in_equilibrium": [("Vap", "Liq")],
    "phase_equilibrium_state": {("Vap", "Liq"): SmoothVLE},
    "bubble_dew_method": LogBubbleDew,
    "parameter_data": {"VDW_kappa": {("benzene", "benzene"): 0.000,
                                    ("benzene", "toluene"): 0.000,
                                    ("toluene", "benzene"): 0.000,
                                    ("toluene", "toluene"): 0.000}}}

### 2.4 Building the model

In the next cell, we will first create a model and attach a the property package. 

In [8]:
def PR_model(data):
    
    m = ConcreteModel()
    
    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(368)
    m.fs.state_block.pressure.fix(101325)
    m.fs.state_block.mole_frac_comp["benzene"].fix(0.5)
    m.fs.state_block.mole_frac_comp["toluene"].fix(0.5)
    
    m.fs.properties.VDW_kappa['benzene', 'toluene'].fix(-0.551198)
    m.fs.properties.VDW_kappa['toluene', 'benzene'].fix(-1.14495)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
       
    # Set bounds on variables to be estimated
    m.fs.properties.VDW_kappa['benzene', 'toluene'].setlb(-3)
    m.fs.properties.VDW_kappa['benzene', 'toluene'].setub(3)

    m.fs.properties.VDW_kappa['toluene', 'benzene'].setlb(-3)
    m.fs.properties.VDW_kappa['toluene', 'benzene'].setub(3)
  
    # Return initialized flash model
    return m

In [9]:
import pytest

test_data = {"temperature": 368}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2022-02-22 13:22:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 0


In [10]:
variable_name = ["fs.properties.VDW_kappa['benzene', 'toluene']",
                "fs.properties.VDW_kappa['toluene', 'benzene']"]

In [11]:
def SSE(m, data):
    expr = ((float(data["vap_benzene"]) -
             m.fs.state_block.mole_frac_phase_comp["Vap", "benzene"])**2 +
            (float(data["liq_benzene"]) -
             m.fs.state_block.mole_frac_phase_comp["Liq", "benzene"])**2)
    return expr*1E4

In [12]:
# solver_options = {'tol': 1e-7}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

obj_value, parameters = pest.theta_est()

2022-02-22 13:22:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-22 13:22:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-22 13:23:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-22 13:23:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-22 13:23:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-22 13:23:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-22 13:23:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-22 13:23:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-22 13:23:14 [INFO] idaes

   2  3.9237086e+01 3.90e+03 1.41e+02  -1.0 1.29e+01    -  2.66e-01 8.08e-03h  1
   3  3.9229391e+01 3.90e+03 2.81e+03  -1.0 3.44e+02    -  4.30e-02 6.65e-04h  1
   4  3.9313332e+01 3.90e+03 5.22e+03  -1.0 3.92e+02    -  4.17e-02 3.30e-03f  6
   5  3.9818085e+01 3.90e+03 7.82e+03  -1.0 3.97e+02    -  6.10e-02 3.15e-03f  6
   6  4.0105819e+01 3.90e+03 4.31e+03  -1.0 3.47e+02    -  2.80e-02 2.47e-04h  7
   7r 4.0105819e+01 3.90e+03 1.00e+03   0.5 0.00e+00   0.0 0.00e+00 3.25e-07R 13
   8r 4.0473261e+01 1.54e+03 9.91e+02   0.5 2.15e+03    -  4.21e-02 1.11e-03f  1
   9  4.0467668e+01 1.54e+03 3.31e+03  -1.0 1.35e+02    -  2.82e-02 1.09e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  3.5878708e+01 1.54e+03 1.33e+04  -1.0 3.31e+02    -  1.42e-02 1.07e-02f  2
  11  3.6085665e+01 1.53e+03 1.78e+04  -1.0 4.51e+02    -  8.21e-03 4.18e-03f  3
  12  3.6330377e+01 1.53e+03 1.79e+04  -1.0 4.12e+02    -  4.62e-03 6.20e-04h  5
  13  3.6546292e+01 1.53e+03

  95  4.1104359e+01 1.69e+03 6.71e+15  -1.0 4.82e+01    -  1.85e-01 6.67e-02h  2
  96  4.9914315e+01 2.11e+03 3.11e+15  -1.0 4.34e+01    -  8.28e-03 5.57e-02H  1
  97  5.2710598e+01 2.08e+03 2.30e+15  -1.0 3.93e+01    -  7.10e-02 1.50e-02h  1
  98  5.2740328e+01 2.08e+03 4.61e+15  -1.0 3.82e+01    -  5.15e-02 1.42e-04h  1
  99r 5.2740328e+01 2.08e+03 1.00e+03   0.1 0.00e+00    -  0.00e+00 3.56e-07R  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100r 5.3531185e+01 1.05e+03 1.01e+03   0.1 9.61e+02    -  4.34e-03 1.17e-03f  1
 101  5.3540704e+01 1.05e+03 8.83e+01  -1.0 1.95e+01    -  1.24e-03 9.05e-05h  1
 102  5.9127442e+01 1.01e+03 2.71e+05  -1.0 2.15e+01    -  9.05e-05 4.69e-02h  1
 103  5.9130535e+01 1.01e+03 2.71e+05  -1.0 8.22e-01   3.5 3.22e-03 2.79e-04h  1
 104r 5.9130535e+01 1.01e+03 1.00e+03  -0.4 0.00e+00   3.9 0.00e+00 4.66e-07R  4
 105r 6.2348959e+01 4.75e+02 1.00e+03  -0.4 2.67e+02    -  1.14e-03 1.30e-03f  1
 106  7.3490302e+01 4.30e+02

 188  2.4896180e+01 4.18e-07 5.16e-02  -8.6 1.02e-04    -  9.93e-01 1.00e+00h  1
 189  2.4896180e+01 4.07e-07 8.77e-08  -8.6 3.69e-09    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190  2.4896180e+01 4.98e-07 1.43e-08  -8.6 3.17e-09    -  1.00e+00 1.00e+00h  1
 191  2.4896180e+01 4.26e-07 1.12e-08  -9.0 9.00e-08    -  1.00e+00 1.00e+00h  1
 192  2.4896180e+01 4.27e-07 9.79e-09  -9.0 3.47e-09    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 192

                                   (scaled)                 (unscaled)
Objective...............:   2.4896180109019500e+01    2.4896180109019500e+01
Dual infeasibility......:   9.7854905847488166e-09    9.7854905847488166e-09
Constraint violation....:   1.9172918624467770e-10    4.2740430217236280e-07
Complementarity.........:   9.0909244580747835e-10    9.0909244580747835e-10
Overall NLP error.......:   9.7854905847488166e-09    4.2740430217236280e-07


Number of objective func

In [13]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 24.896180

The values for the parameters are as follows:
fs.properties.VDW_kappa[benzene,toluene] = -0.5489910588940975
fs.properties.VDW_kappa[toluene,benzene] = -1.1654249247353858
